# OPS-SAT case starter-kit notebook

ESA's [Kelvins](https://kelvins.esa.int) competition "[the OPS-SAT case](https://kelvins.esa.int/opssat/home/)" is a novel data-centric challenge that asks you to work with the raw data of a satellite and very few provided labels to find the best parameters for a given machine learning model. Compared to previous competitions on Kelvins (like the [Pose Estimation](https://kelvins.esa.int/pose-estimation-2021/) or the [Proba-V Super-resolution challenge](https://kelvins.esa.int/proba-v-super-resolution/)) where the test-set is provided and the infered results are submitted, for the OPS-SAT case, we will run inference on the Kelvins server directly! This notebooks contains examples on how you can load your data and train an **EfficientNetLite0** model by only using the 80-labeled images provided. Therefore, the directory `images`, containing unlabeld patches and included in the training dataset is not used for this notebook. However, competitors are encouraged to use these patches to improve the model accuracy.

# 1. Module imports

If you do not have a GPU, uncomment and run the next commands.


In [52]:
import sys
sys.version

'3.9.16 (main, Mar  8 2023, 14:00:05) \n[GCC 11.2.0]'

In [53]:
! python --version

/usr/bin/zsh: /home/ramez/miniconda3/envs/thesis/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
Python 3.9.16


In [54]:
! which python

/usr/bin/zsh: /home/ramez/miniconda3/envs/thesis/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
/home/ramez/miniconda3/envs/thesis/bin/python


In [55]:
import tensorflow as tf

#### GPU Check

In [56]:
!python3 -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"

/usr/bin/zsh: /home/ramez/miniconda3/envs/thesis/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [57]:
from tensorflow import keras

In [58]:
import numpy as np

In [59]:
from sklearn.metrics import cohen_kappa_score

In [60]:
## Local EfficientNetLite (Customized by the Competition)
from efficientnet_lite import EfficientNetLiteB0

In [61]:
from sklearn.model_selection import KFold

In [62]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

In [63]:
from tensorflow.keras.callbacks import TensorBoard

# 2. Utility Functions

You can use this function to load your training data.

In [64]:
def get_images_from_path(dataset_path):
    """ Get images from path and normalize them applying channel-level normalization. """

    # loading all images in one large batch
    tf_eval_data = tf.keras.utils.image_dataset_from_directory(dataset_path, image_size=input_shape[:2], shuffle=False, 
                                                               batch_size=100000)

    # extract images and targets
    for tf_eval_images, tf_eval_targets in tf_eval_data:
        break

    return tf.convert_to_tensor(tf_eval_images), tf_eval_targets

# 3. Loading the model

The network architecture used for OPS-SAT is **EfficientNetLite0**. We would like to thank Sebastian for making a Keras implementation of EfficientNetLite publicly available under the Apache 2.0 License: https://github.com/sebastian-sz/efficientnet-lite-keras. Our Version of this code has been modified to better fit our purposes. For example, we removed the ReLU "stem_activation" to better match a related efficientnet pytorch implementation. In any way, **you have to use the model architecture that we provide in our [starter-kit](https://gitlab.com/EuropeanSpaceAgency/the_opssat_case_starter_kit).**

In [65]:
input_shape = (200, 200, 3)   # input_shape is (height, width, number of channels) for images
num_classes = 8

## Load The Model Without any Weights

In [66]:
# model = EfficientNetLiteB0(classes=num_classes, weights=None, input_shape=input_shape, classifier_activation=None)
# model.summary()

## Load The Model With ImageNet Weights

In [67]:
model = EfficientNetLiteB0(classes=num_classes, weights='imagenet', input_shape=input_shape, classifier_activation=None, include_top = False)
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
output_layer = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=model.input, outputs=output_layer)
# model.summary()

block1a_ same
block2a_ ((1, 1), (1, 1))
block2a_ valid
block2b_ same
block3a_ ((2, 2), (2, 2))
block3a_ valid
block3b_ same
block4a_ ((1, 1), (1, 1))
block4a_ valid
block4b_ same
block4c_ same
block5a_ same
block5b_ same
block5c_ same
block6a_ ((2, 2), (2, 2))
block6a_ valid
block6b_ same
block6c_ same
block6d_ same
block7a_ same


# 4. Loading data

In [68]:
# dataset_path_train="/home/ramez/Politechnika_Slaska_MSc/Thesis/Competition/Data/images_copy_processed/"
dataset_path_train_val = "/home/ramez/Politechnika_Slaska_MSc/Thesis/Competition/Data/ops_sat_train/"

In [69]:
dataset_path_test = "/home/ramez/Politechnika_Slaska_MSc/Thesis/Competition/Data/ops_sat_test/"

In this notebook, classical supervised learning is used. Therefore, remember to remove the subdirectory `images` containing unlabeled patches before loading the dataset to perform training correctly.

In [70]:
#Loading dataset
x_train_val, y_train_val = get_images_from_path(dataset_path_train_val)

Found 227 files belonging to 8 classes.


In [71]:
#Loading dataset
x_test, y_test = get_images_from_path(dataset_path_test)

Found 40 files belonging to 8 classes.


# 5. Model training

We provide now an example on how you can train your model by using standard supervised learning. Training loss (`SparseCategoricalCrossentropy`) and `Accuracy` are shown for simplicity and for an easier interpretation of the training outcome, despite your submission will be evaluated by using the metric **1 - Cohen's kappa** [metric](https://en.wikipedia.org/wiki/Cohen's_kappa). For more information on scoring, please refer to [Scoring](https://kelvins.esa.int/opssat/scoring/).

With this model and the dataset provided, please do your best!

### Compile The model

In [72]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

### Early Stopping Callback

In [87]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

### Without K fold Cross Validation but with TensorBoard (With Early Stopping)

In [74]:
# tensorboard_callback = TensorBoard(log_dir='./logs')

In [75]:
# early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
# history = model.fit(x_train_val, y_train_val, validation_data=(x_test, y_test), epochs= 100, verbose=1, batch_size=8, 
#                         callbacks=[early_stopping, tensorboard_callback])

In [76]:
# !tensorboard --logdir=./logs

### With K fold Cross Validation but without TensorBoard (With Early Stopping)

In [77]:
k = 5
kf = KFold(n_splits=k, shuffle=True)

In [84]:
# Train and evaluate the model using K-fold cross-validation
scores = []
for train_idx, val_idx in kf.split(x_train_val):
    X_train = tf.gather(x_train_val, train_idx)
    y_train = tf.gather(y_train_val, train_idx)
    X_val = tf.gather(x_train_val, val_idx)
    y_val = tf.gather(y_train_val, val_idx)
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs= 100, verbose=1, batch_size=8, 
                        callbacks=[early_stopping])

    score = model.evaluate(X_val, y_val)
    scores.append(score[1])

Epoch 1/100
23/23 [==============================] - 3s 120ms/step - loss: 0.2388 - sparse_categorical_accuracy: 0.9171 - val_loss: 0.0641 - val_sparse_categorical_accuracy: 0.9783
Epoch 2/100
23/23 [==============================] - 3s 118ms/step - loss: 0.1480 - sparse_categorical_accuracy: 0.9669 - val_loss: 0.4565 - val_sparse_categorical_accuracy: 0.9348
Epoch 3/100
23/23 [==============================] - 3s 123ms/step - loss: 0.1172 - sparse_categorical_accuracy: 0.9724 - val_loss: 0.0894 - val_sparse_categorical_accuracy: 0.9783
Epoch 4/100
23/23 [==============================] - 3s 123ms/step - loss: 0.0543 - sparse_categorical_accuracy: 0.9779 - val_loss: 0.0173 - val_sparse_categorical_accuracy: 1.0000
Epoch 5/100
23/23 [==============================] - 3s 118ms/step - loss: 0.0316 - sparse_categorical_accuracy: 0.9890 - val_loss: 0.0263 - val_sparse_categorical_accuracy: 1.0000
Epoch 6/100
23/23 [==============================] - 3s 124ms/step - loss: 0.0183 - sparse_cate

In [85]:
# Print the mean validation accuracy
print('Validation accuracy mean: {:.2f} (std {:.2f})'.format(np.mean(scores), np.std(scores)))

Validation accuracy mean: 0.97 (std 0.03)


Calculating the **1 - Cohen's kappa** score of the trained model on the trained dataset.

In [86]:
predictions = np.zeros(len(y_test), dtype=np.int8)
# inference loop
for e, (image, target) in enumerate(zip(x_test, y_test)):
    image = np.expand_dims(np.array(image), axis=0)
    output = model.predict(image)
    predictions[e] = np.squeeze(output).argmax()
#Keras model score
score_keras = 1 - cohen_kappa_score(y_test.numpy(), predictions)
print("Score:",score_keras)

Score: 0.6571428571428571


# 6. Saving and loading trained model

The trained model can be now saved by using HDF5-format that is the only accepted for submission. The name `test.h5` will be used.

In [ ]:
#Saving model
# model.save_weights('test.h5')
model.save_weights('/home/ramez/Politechnika_Slaska_MSc/Thesis/Competition/submission/submit_19.h5')

The trained model can be also loaded for further testing. 

In [ ]:
model = EfficientNetLiteB0(classes=num_classes, weights=None, input_shape=input_shape, classifier_activation=None)
# model = EfficientNetLiteB0(include_top=False, classes=num_classes, weights='imagenet', input_shape=input_shape, classifier_activation=None)
model.load_weights('/home/ramez/Politechnika_Slaska_MSc/Thesis/Competition/submission/model_patterns_20epochs.h5')

The model will be now compiled and tested again. You should get the same score as before saving and loading. 

In [ ]:
#Model shall be compiled before testing.
model.compile()

#Creating empty predictions
predictions = np.zeros(len(y_train), dtype=np.int8)

# inference loop
for e, (image, target) in enumerate(zip(x_train, y_train)):
    image = np.expand_dims(np.array(image), axis=0)
    output = model.predict(image)
    predictions[e] = np.squeeze(output).argmax()

#Keras model score
score_keras = 1 - cohen_kappa_score(y_train.numpy(), predictions)
print("Score:",score_keras)